In [4]:
import sys
sys.path.append('..')

import constants.file_handler_constants as fh
from constants.user_constants import *
from constants.attraction_constants import *

import os
import glob
import time
import pandas as pd
import numpy as np
import json
import requests
import ast

import google.generativeai as genai
from google.generativeai.types import ContentType
from PIL import Image
from IPython.display import Markdown

In [5]:
def getScorefromGeminiAPI(user_preference_prompt: list[dict]) -> dict:
    """
    Retrieves tag scores for a user based on their responses during the sign-up process using the Gemini API.

    Args:
        user_preference_prompt : A list of the user's responses to each question

    Returns:
        Dictionary representing scores for all tags.
    """

    # configure the Gemini API model
    genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
    model = genai.GenerativeModel('gemini-1.5-flash-latest')
    # model = genai.GenerativeModel('gemini-1.5-pro-latest')
    
    # construct the base prompt
    text_prompt = (
        "Based on the user's answers, generate a JSON string containing scores (0-1) for the following attributes (nothing else, no other sentences):"
        "\nExample JSON: "
        '{"Tourism":0,"Adventure":0,"Meditation":0,"Art":0,"Cultural":0,"Landscape":0,"Nature":0,"Historical":0,'
        '"Cityscape":0,"Beach":0,"Mountain":0,"Architecture":0,"Temple":0,"WalkingStreet":0,"Market":0,'
        '"Village":0,"NationalPark":0,"Diving":0,"Snuggle":0,"Waterfall":0,"Island":0,"Shopping":0,'
        '"Camping":0,"Fog":0,"Cycling":0,"Monument":0,"Zoo":0,"Waterpark":0,"Hiking":0,"Museum":0,'
        '"Riverside":0,"NightLife":0,"Family":0,"Kid":0,"Landmark":0,"Forest":0}'
    )

    # Add each question and its corresponding score to the prompt
    for item in user_preference_prompt:
        question = item["question"]
        score = item["score"]
        text_prompt += f"\nQuestion: {question}\nUser Answer: {score}/5"

    # 
    prompt = [text_prompt]

    # print token count for debugging
    print("total_tokens: ", model.count_tokens(prompt))
    
    # send the prompt to the Gemini API
    res_score_dict = {}
    try:
        response = model.generate_content(prompt)
        res_start_Idx = response.text.find('{')
        res_end_Idx = response.text.find('}')
        res_score_dict =  json.loads(response.text[res_start_Idx:res_end_Idx+1])

    except Exception as e:
        print("failed to use gemini api")
    
    return res_score_dict

In [6]:
user_preference_prompt = [
    {"name": "Nature & Outdoors", "score": 0, 
    "question": "On a scale of 0-5, how much do you enjoy spending time in nature, such as mountains, beaches, national parks, waterfalls, forests, or riversides?"},

    {"name": "Cultural & Historical", "score": 0, 
    "question": "On a scale of 0-5, how much do you enjoy exploring cultural and historical attractions like temples, museums, landmarks, or monuments?"},

    {"name": "Urban & City Life", "score": 0, 
    "question": "On a scale of 0-5, how much do you enjoy urban experiences like shopping, walking streets, cityscapes, markets, or nightlife?"},

    {"name": "Relaxation & Meditation", "score": 0, 
     "question": "On a scale of 0-5, how much do you enjoy relaxing activities like meditation, cozy snuggles, or peaceful foggy settings?"},

    {"name": "Adventure & Activities", "score": 3, 
     "question": "On a scale of 0-5, how much do you enjoy adventure activities like hiking, cycling, camping, or diving?"},

    {"name": "Wildlife & Family-Friendly", "score": 0, 
     "question": "On a scale of 0-5, how much do you enjoy visiting family-friendly places like zoos or waterparks with activities suitable for kids?"},

    {"name": "Arts & Architecture", "score": 5, 
     "question": "On a scale of 0-5, how much do you enjoy exploring art and architecture during your travels?"},

    {"name": "Small Villages & Rural Life", "score": 0, 
     "question": "On a scale of 0-5, how much do you enjoy visiting small villages and experiencing rural life?"},

    {"name": "Islands & Beaches", "score": 0, 
     "question": "On a scale of 0-5, how much do you enjoy visiting islands and spending time on beaches?"},
]


user_attraction_tag_score = getScorefromGeminiAPI(user_preference_prompt=user_preference_prompt.copy())
user_attraction_tag_score

total_tokens:  total_tokens: 532



{'Tourism': 0,
 'Adventure': 0.6,
 'Meditation': 0,
 'Art': 1,
 'Cultural': 0,
 'Landscape': 0,
 'Nature': 0,
 'Historical': 0,
 'Cityscape': 0,
 'Beach': 0,
 'Mountain': 0,
 'Architecture': 1,
 'Temple': 0,
 'WalkingStreet': 0,
 'Market': 0,
 'Village': 0,
 'NationalPark': 0,
 'Diving': 0.6,
 'Snuggle': 0,
 'Waterfall': 0,
 'Island': 0,
 'Shopping': 0,
 'Camping': 0.6,
 'Fog': 0,
 'Cycling': 0.6,
 'Monument': 0,
 'Zoo': 0,
 'Waterpark': 0,
 'Hiking': 0.6,
 'Museum': 0,
 'Riverside': 0,
 'NightLife': 0,
 'Family': 0,
 'Kid': 0,
 'Landmark': 0,
 'Forest': 0}